In [1]:
%matplotlib inline
# Load all necessary packages
import sys
import pandas as pd
import matplotlib.pyplot as plt
# Import IBM's AI Fairness tooolbox
from aif360.datasets import BinaryLabelDataset
from aif360.metrics  import BinaryLabelDatasetMetric
from aif360.metrics  import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
from aif360.algorithms.preprocessing.lfr import LFR
# Import scikit-learn core slibraries
from sklearn.ensemble      import RandomForestClassifier
from sklearn.linear_model  import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics       import accuracy_score
from IPython.display import Markdown, display

# Warnings will be used to silence various model warnings for tidier output
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read the cleaned Adult dataset
Adult_df = pd.read_csv('./input/adult-cleaned.csv')
# The AIF demo drops the following columns - we'll try the same
Adult_df.drop(["Fnlwgt", "NativeCountry", "Relationship", "MaritalStatus"],axis=1,inplace=True)

In [3]:
# Set privileged (1)/ unprivileged (0)/ favourable (1) / unfavourable values (0)
protected_attr      = 'Gender'
priv_grp            = 1  # Males 
unpriv_grp          = 0  # Females  
lab                 = 'Income'
fav_label           = 1 # Income over £50K
unfav_label         = 0 # Income under £50K
privileged_groups   = [{protected_attr: priv_grp}]   # Males
unprivileged_groups = [{protected_attr: unpriv_grp}] # Females

In [4]:
# Create a Binary Label Dataset to use with AIF360 APIs
X = Adult_df.drop(lab,axis=1)
y = Adult_df[lab]

In [5]:
Adult_bld = BinaryLabelDataset(df=pd.concat((X, y), axis=1),
                                label_names=[lab], protected_attribute_names=[protected_attr],
                                favorable_label=fav_label, unfavorable_label=unfav_label)

In [6]:
# Create train and test datasets
Adult_train_df, Adult_test_df = Adult_bld.split([0.8], shuffle=True, seed=101)

In [7]:
scaler = MinMaxScaler(copy=False)
Adult_train_df.features = scaler.fit_transform(Adult_train_df.features)
Adult_test_df.features  = scaler.fit_transform(Adult_test_df.features)

In [8]:
# Determine the baseline model accuracy for Logistic Regression and Random Forest Classifiers
Adult_train, d = Adult_train_df.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
Adult_test, d = Adult_test_df.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
X_train = Adult_train.drop(lab,axis=1)
y_train = Adult_train[lab]
X_test = Adult_test.drop(lab,axis=1)
y_test = Adult_test[lab]
biasedLogModel = LogisticRegression(random_state=101)
biasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
biasedLogModel.fit(X_train, y_train) 
biasedRfcModel.fit(X_train, y_train) 
print(f"Logistic regression validation accuracy: {biasedLogModel.score(X_test, y_test)}")
print(f"Random Forest       validation accuracy: {biasedRfcModel.score(X_test, y_test)}")

Logistic regression validation accuracy: 0.8207595455010749
Random Forest       validation accuracy: 0.8296652676834886


In [9]:
# Create the binary label dataset metric class for the training dataset
metric_train_bld = BinaryLabelDatasetMetric(Adult_train_df, 
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)
display(Markdown("#### Biased training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_train_bld.mean_difference())
print('Number of instances           :', metric_train_bld.num_instances())
print("Base Rate                     :%f" % metric_train_bld.base_rate())
print('Consistency                   :', metric_train_bld.consistency())
print('Disparate Impact              :', metric_train_bld.disparate_impact())
print('Mean Difference               :', metric_train_bld.mean_difference())
print('Statistical Parity Difference :', metric_train_bld.statistical_parity_difference()) 
print('# of positives(privileged)    :', metric_train_bld.num_positives(privileged=True))
print('# of positives(non-privileged):', metric_train_bld.num_positives(privileged=False))
print('Total positive instances"     :', metric_train_bld.num_positives(privileged=True)+metric_train_bld.num_positives(privileged=False))
print('# of negatives(privileged)    :', metric_train_bld.num_negatives(privileged=True))
print('# of negatives(non-privileged):', metric_train_bld.num_negatives(privileged=False))
print('Total negative instances"     :', metric_train_bld.num_negatives(privileged=True)+metric_train_bld.num_negatives(privileged=False))
display(Markdown("#### Biased training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_train_bld.mean_difference())

#### Biased training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.192939
Number of instances           : 39073.0
Base Rate                     :0.238605
Consistency                   : [0.81125585]
Disparate Impact              : 0.3620832314871518
Mean Difference               : -0.19293913814906824
Statistical Parity Difference : -0.19293913814906824
# of positives(privileged)    : 7907.0
# of positives(non-privileged): 1416.0
Total positive instances"     : 9323.0
# of negatives(privileged)    : 18236.0
# of negatives(non-privileged): 11514.0
Total negative instances"     : 29750.0


#### Biased training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.192939


In [10]:
# Fit the Learning Fair Representations on the biased training data
print('Interval : Optimization objective value for the interval')
TR = LFR(unprivileged_groups = unprivileged_groups, privileged_groups = privileged_groups,verbose=1, seed=101)
TR = TR.fit(Adult_train_df)

Interval : Optimization objective value for the interval
250 20315.451767096565
500 19286.966322690554
750 18408.53142080968
1000 17549.43702966024
1250 17124.241896159663
1500 17025.29486604778
1750 16808.634902034868
2000 16574.82065876097
2250 16542.584425679714
2500 16445.904068983964
2750 16353.399057267265
3000 16301.11574492851
3250 16255.995310472583
3500 16186.452308296375
3750 16151.240155774334
4000 16099.369041745846
4250 16091.432413938335
4500 16059.342582387313
4750 16043.460498825825
5000 15999.343402532852


In [11]:
# Transform training data and align features
Adult_train_lfr = TR.transform(Adult_train_df)

In [12]:
# Determine the transformed model accuracy for Logistic Regression and Random Forest Classifiers
# Convert the scaled Binary Labelled Dataset to a pandas dataframe for consistency 
Adult_train, d = Adult_train_lfr.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
X_train = Adult_train.drop(lab,axis=1)
y_train = Adult_train[lab]
debiasedLogModel = LogisticRegression(random_state=101)
debiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
debiasedLogModel.fit(X_train, y_train) 
debiasedRfcModel.fit(X_train, y_train) 
print(f"Logistic regression validation accuracy: {debiasedLogModel.score(X_test, y_test)}")
print(f"Random Forest       validation accuracy: {debiasedRfcModel.score(X_test, y_test)}")

Logistic regression validation accuracy: 0.7848295629030607
Random Forest       validation accuracy: 0.7558603746545194


In [13]:
display(Markdown("#### Transformed training dataset"))
from sklearn.metrics import classification_report
thresholds = [0.1, 0.3, 0.35, 0.4, 0.5, 0.6, 0.7]
for threshold in thresholds:
    
    # Transform training data and align features
    Adult_train_lfr = TR.transform(Adult_train_df,threshold=threshold)
    metric_train_lfr = BinaryLabelDatasetMetric(Adult_train_lfr, 
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
    print("Classification threshold = %f" % threshold)
    #print(classification_report(dataset_orig_train.labels, dataset_transf_train.labels))
    print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_train_lfr.mean_difference())

#### Transformed training dataset

Classification threshold = 0.100000
Difference in mean outcomes between unprivileged and privileged groups = -0.444327
Classification threshold = 0.300000
Difference in mean outcomes between unprivileged and privileged groups = -0.349896
Classification threshold = 0.350000
Difference in mean outcomes between unprivileged and privileged groups = -0.301159
Classification threshold = 0.400000
Difference in mean outcomes between unprivileged and privileged groups = -0.260219
Classification threshold = 0.500000
Difference in mean outcomes between unprivileged and privileged groups = -0.184895
Classification threshold = 0.600000
Difference in mean outcomes between unprivileged and privileged groups = -0.125776
Classification threshold = 0.700000
Difference in mean outcomes between unprivileged and privileged groups = -0.079608


In [14]:
print('THIS IS WHAT THE AIF CODE PRODUCES FROM THE LAST CLASSIFICATION THRESHOLD RUN ABOVE')
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_train_lfr.mean_difference())
print('Number of instances           :', metric_train_lfr.num_instances())
print("Base Rate                     :%f" % metric_train_lfr.base_rate())
print('Consistency                   :', metric_train_lfr.consistency())
print('Disparate Impact              :', metric_train_lfr.disparate_impact())
print('Mean Difference               :', metric_train_lfr.mean_difference())
print('Statistical Parity Difference :', metric_train_lfr.statistical_parity_difference()) 
print('# of positives(privileged)    :', metric_train_lfr.num_positives(privileged=True))
print('# of positives(non-privileged):', metric_train_lfr.num_positives(privileged=False))
print('Total positive instances"     :', metric_train_lfr.num_positives(privileged=True)+metric_train_lfr.num_positives(privileged=False))
print('# of negatives(privileged)    :', metric_train_lfr.num_negatives(privileged=True))
print('# of negatives(non-privileged):', metric_train_lfr.num_negatives(privileged=False))
print('Total negative instances"     :', metric_train_lfr.num_negatives(privileged=True)+metric_train_lfr.num_negatives(privileged=False))
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_train_bld.mean_difference())

THIS IS WHAT THE AIF CODE PRODUCES FROM THE LAST CLASSIFICATION THRESHOLD RUN ABOVE
Difference in mean outcomes between unprivileged and privileged groups = -0.079608
Number of instances           : 39073.0
Base Rate                     :0.059683
Consistency                   : [0.99987203]
Disparate Impact              : 0.07461833170160356
Mean Difference               : -0.07960767211119969
Statistical Parity Difference : -0.07960767211119969
# of positives(privileged)    : 2249.0
# of positives(non-privileged): 83.0
Total positive instances"     : 2332.0
# of negatives(privileged)    : 23894.0
# of negatives(non-privileged): 12847.0
Total negative instances"     : 36741.0


#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.192939


In [15]:
display(Markdown("#### Individual fairness metrics"))
print("Consistency of labels in transformed training dataset= %f" %metric_train_lfr.consistency())
print("Consistency of labels in original training dataset= %f" %metric_train_bld.consistency())

#### Individual fairness metrics

Consistency of labels in transformed training dataset= 0.999872
Consistency of labels in original training dataset= 0.811256


In [16]:
# The following Cells compare the original and transformed datasets
#Convert the returned Binary Labelled Dataset to a pandas dataframe 
Adult_orig_df, d = Adult_train_df.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
Adult_lfr_df, d = Adult_train_lfr.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
# Check whether the transform on the original dataset has worked. 
# A false means that the dataset is transformed.
Adult_lfr_df.equals(Adult_df)

False

In [17]:
Adult_lfr_df.head(3)

,Age,Employment,Education,EducationNum,Occupation,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,Income
30658,0.083403,0.206938,0.270332,0.342235,0.324137,0.204671,0.0,-0.963563,-0.153807,0.364831,0.0
25493,0.070238,0.198302,0.275690,0.317222,0.315824,0.200837,1.0,-1.042219,-0.176152,0.345351,0.0
11136,0.067842,0.196599,0.276738,0.312645,0.314252,0.200134,0.0,-1.056612,-0.180320,0.341817,0.0


In [18]:
Adult_orig_df.head(3)

,Age,Employment,Education,EducationNum,Occupation,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,Income
30658,0.068493,0.0,0.133333,0.733333,0.461538,0.25,0.0,0.0,0.0,0.265306,0.0
25493,0.082192,0.5,0.066667,0.533333,0.230769,0.25,1.0,0.0,0.0,0.397959,0.0
11136,0.287671,0.0,0.066667,0.533333,0.461538,0.25,0.0,0.0,0.0,0.448980,0.0


In [19]:
Adult_orig_df   = Adult_orig_df.reset_index(drop=True)
Adult_lfr_df = Adult_lfr_df.reset_index(drop=True)
AdultBool   = (Adult_orig_df != Adult_lfr_df).stack()  # Create Frame of comparison booleans
Adultdiff   = pd.concat([Adult_orig_df.stack()[AdultBool], Adult_lfr_df.stack()[AdultBool]], axis=1)
Adultdiff.columns=["Adult_df", "Adult_lfr"]
print(Adultdiff)

                    Adult_df  Adult_lfr
0     Age           0.068493   0.083403
      Employment    0.000000   0.206938
      Education     0.133333   0.270332
      EducationNum  0.733333   0.342235
      Occupation    0.461538   0.324137
      Race          0.250000   0.204671
      CapitalGain   0.000000  -0.963563
      CapitalLoss   0.000000  -0.153807
      HoursPerWeek  0.265306   0.364831
1     Age           0.082192   0.070238
      Employment    0.500000   0.198302
      Education     0.066667   0.275690
      EducationNum  0.533333   0.317222
      Occupation    0.230769   0.315824
      Race          0.250000   0.200837
      CapitalGain   0.000000  -1.042219
      CapitalLoss   0.000000  -0.176152
      HoursPerWeek  0.397959   0.345351
2     Age           0.287671   0.067842
      Employment    0.000000   0.196599
      Education     0.066667   0.276738
      EducationNum  0.533333   0.312645
      Occupation    0.461538   0.314252
      Race          0.250000   0.200134


In [20]:
## PCA Analysis of consitency

In [21]:
# At this stage the transformed dataframe will have the last threshold encountered!
print('this is the threshold being used :',threshold)
feat_cols = Adult_train_df.feature_names

orig_df = pd.DataFrame(Adult_train_df.features,columns=feat_cols)
orig_df['label'] = Adult_train_df.labels
orig_df['label'] = orig_df['label'].apply(lambda i: str(i))

transf_df = pd.DataFrame(Adult_train_lfr.features,columns=feat_cols)
transf_df['label'] = Adult_train_lfr.labels
transf_df['label'] = transf_df['label'].apply(lambda i: str(i))

this is the threshold being used : 0.7


In [22]:
from sklearn.decomposition import PCA

orig_pca = PCA(n_components=3)
orig_pca_result = orig_pca.fit_transform(orig_df[feat_cols].values)

orig_df['pca-one'] = orig_pca_result[:,0]
orig_df['pca-two'] = orig_pca_result[:,1] 
orig_df['pca-three'] = orig_pca_result[:,2]

display(Markdown("#### Original training dataset"))
print('Explained variation per principal component:')
print(orig_pca.explained_variance_ratio_)

#### Original training dataset

Explained variation per principal component:
[0.46080807 0.12983659 0.10831626]


In [23]:
transf_pca = PCA(n_components=3)
transf_pca_result = transf_pca.fit_transform(transf_df[feat_cols].values)

transf_df['pca-one'] = transf_pca_result[:,0]
transf_df['pca-two'] = transf_pca_result[:,1] 
transf_df['pca-three'] = transf_pca_result[:,2]

display(Markdown("#### Transformed training dataset"))
print('Explained variation per principal component:')
print(transf_pca.explained_variance_ratio_)

#### Transformed training dataset

Explained variation per principal component:
[9.99520929e-01 4.79071361e-04 1.30617896e-13]


In [24]:
# Test whether we can predict the Sensitive (Protected) variable from the transformed training dataset
X_train = Adult_lfr_df.drop(protected_attr,axis=1)
y_train = Adult_lfr_df[protected_attr]
debiasedLogModel = LogisticRegression(random_state=101)
debiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
debiasedLogModel.fit(X_train, y_train) 
debiasedRfcModel.fit(X_train, y_train) 
# Now test whether we can predict Gender from the test dataset
Adult_test, d = Adult_test_df.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
X_test = Adult_test.drop(protected_attr,axis=1)
y_test = Adult_test[protected_attr]
print(f"Logistic regression validation accuracy: {debiasedLogModel.score(X_test, y_test)}")
print(f"Random Forest       validation accuracy: {debiasedRfcModel.score(X_test, y_test)}")

Logistic regression validation accuracy: 0.5037363087317023
Random Forest       validation accuracy: 0.36605589108404135


In [25]:
###Load, clean up original test data and compute metric

In [26]:
display(Markdown("#### Testing Dataset shape"))
print(Adult_test_df.features.shape)

metric_test_bld = BinaryLabelDatasetMetric(Adult_test_df, 
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)
display(Markdown("#### Original test dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_test_bld.mean_difference())

#### Testing Dataset shape

(9769, 10)


#### Original test dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.200836


In [27]:
###Transform test data and compute metric

In [28]:
Adult_test_lfr = TR.transform(Adult_test_df, threshold=threshold)
metric_test_lfr = BinaryLabelDatasetMetric(Adult_test_lfr, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)

In [29]:
print("Consistency of labels in tranformed test dataset= %f" %metric_test_lfr.consistency())

Consistency of labels in tranformed test dataset= 0.999672


In [30]:
print("Consistency of labels in original test dataset= %f" %metric_test_bld.consistency())

Consistency of labels in original test dataset= 0.803890


In [31]:
def check_algorithm_success():
    """Transformed dataset consistency should be greater than original dataset."""
    assert metric_test_lfr.consistency() > metric_test_bld.consistency(), "Transformed dataset consistency should be greater than original dataset."

print(check_algorithm_success())

None
